# 一个简单的网络模型

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义网络
class Net(nn.Module):

    def __init__(self):
        # 继承原有模型
        super(Net, self).__init__()
        
        # 定义了两个卷积层
        # 第一层，输入1维的（单通道，灰色）图片，输出6维的的卷积层（用到了6个卷积核，每个卷积核是5*5的）。
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 第二层
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # 定义了三个全连接层，即fc1与conv2相连，将16张5*5的卷积网络一维化，并输出120个节点。
        self.fc1 = nn.Linear(16 * 5 * 5, 120)        
        # 将120个节点转化为84个。
        self.fc2 = nn.Linear(120, 84)       
        # 将84个节点输出为10个，即有10个分类结果。
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x): 
        # 用relu激活函数作为一个池化层，池化的窗口大小是2*2，与上文的16*5*5的计算结果相符。
        # 先用conv1卷积，然后激活，激活的窗口是2*2。
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))   
        # 作用同上。窗口是正方形的时候，2的写法等同于（2，2）。
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # 调用定义好的函数，将高维的向量转化为一维。
        x = x.view(-1, self.num_flat_features(x))
        
        # 用全连接层fc1和fc2，然后做激活。
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))     
        # 用全连接层fc3。
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        # 承接上文的引用，由于pytorch只接受图片集的输入方式,所以第一个代表个数的维度被忽略。
        size = x.size()[1:]  
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# 创建网络，并查看
net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
